In [1]:
## Import modules

from tqdm import trange

import torch
import torchvision
import torchvision.transforms as transforms

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print("GPU not found, using CPU")

In [2]:
## Load data
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(degrees=45),
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

train_dataset = torchvision.datasets.CIFAR10(
    "data/cifar10", transform=transform, download=True
)

Files already downloaded and verified


In [3]:
## Load model
model = torchvision.models.resnet50(pretrained=True)
model = torch.nn.DataParallel(model)
model = model.to(device)

/home/loek/miniconda3/envs/rnn/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/loek/miniconda3/envs/rnn/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
## Set hyperparameters
n_epochs = 10
batch_size = 64
learning_rate = 0.001

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=2
)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
## Training run
iterator = trange(n_epochs, desc="Training", unit="steps")
for epoch in iterator:
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

    iterator.set_postfix(train_loss="{:.5f}".format(loss.item()))

Training:   0%|          | 0/10 [00:11<?, ?steps/s]


KeyboardInterrupt: 